# Credit Card Optimizer Using Pulp

Name: Engineer Investor (@egr_investor)

Date: 04/11/2024

## Note:

Due to personal circumstances, I currently do not have the bandwidth to launch my credit card optimizer website. However, I still plan to do so in the future. For now, I am releasing my optimization algorithm to the public as open-source under the specified license to allow others to iterate and improve upon it. Please note that I am not sharing my credit card rewards database at this time.

## Install dependencies

In [76]:
!pip install pulp pandas

In [77]:
import pulp
import pandas as pd
from typing import Dict, List, Union, Tuple, Set

## Define algorithm

In [78]:
def optimize_credit_cards(
    cards: Dict[str, Dict[str, float]],
    spending: Dict[str, float],
    annual_fees: Dict[str, float],
    ) -> Union[Tuple[Dict[str, str], float, Set[str]], str]:
    """
    This function optimizes credit card usage to maximize rewards while
    considering spending patterns and annual fees.

    Args:
        cards (dict): Dictionary where keys are card IDs and values are dictionaries
                     containing reward rates for each spending category.
        spending (dict): Dictionary where keys are spending categories and values
                         are the annual amount spent in that category.
        annual_fees (dict): Dictionary where keys are card IDs and values are the
                            annual fees for each card.

    Returns:
        tuple: A tuple containing three elements:
               - chosen_cards (dict): Dictionary mapping spending categories to the
                                      chosen card for that category.
               - total_reward (float): The total reward earned by using the chosen cards.
               - unique_cards (set): Set containing the unique set of cards selected.
               - If no positive reward solution is found, returns a string message
                 indicating this and sets total_reward and unique_cards to 0 and empty set respectively.
    """

    # Create a problem variable to maximize rewards
    prob = pulp.LpProblem("Maximize Credit Card Rewards", pulp.LpMaximize)

    # Decision variables: card_category_vars[card_id][category] = {0, 1}
    card_category_vars = {
        card: {category: pulp.LpVariable(f"card_{card}_{category}", 0, 1, pulp.LpBinary)
               for category in spending}
        for card in cards
    }

    # Objective function: Maximize the total reward minus the annual fees
    rewards = pulp.lpSum([card_category_vars[card][category] * spending[category] * cards[card][category]
                          for card in cards for category in spending])
    fees = pulp.lpSum([card_category_vars[card][category] * annual_fees[card]
                       for card in cards for category in spending])
    total_reward = rewards - fees
    prob += total_reward

    # Constraint: Total reward must be positive
    prob += total_reward >= 0

    # Constraints: Each category can only have one card
    for category in spending:
        prob += pulp.lpSum([card_category_vars[card][category] for card in cards]) == 1

    # Solve the problem
    prob.solve()

    # Check if a positive reward solution exists
    if pulp.LpStatus[prob.status] == 'Optimal':
        # Extract the chosen cards for each category
        chosen_cards = {category: next(card for card in cards if pulp.value(card_category_vars[card][category]) == 1)
                        for category in spending}
        total_reward = pulp.value(prob.objective)

        # Extract the unique set of cards selected
        unique_cards = set(chosen_cards.values())

        return chosen_cards, total_reward, unique_cards
    else:
        return "No positive reward solution found", 0, set()


## Example Usuage

In [79]:
# Expanded Example Usage
cards = {
    'Card1': {'Groceries': 0.03, 'Gas': 0.02, 'Dining': 0.01, 'Travel': 0.04, 'Entertainment': 0.02},
    'Card2': {'Groceries': 0.02, 'Gas': 0.03, 'Dining': 0.02, 'Travel': 0.01, 'Entertainment': 0.03},
    'Card3': {'Groceries': 0.01, 'Gas': 0.04, 'Dining': 0.03, 'Travel': 0.02, 'Entertainment': 0.02},
    'Card4': {'Groceries': 0.04, 'Gas': 0.01, 'Dining': 0.02, 'Travel': 0.03, 'Entertainment': 0.04},
    'Card5': {'Groceries': 0.05, 'Gas': 0.01, 'Dining': 0.01, 'Travel': 0.02, 'Entertainment': 0.03},
    'Card6': {'Groceries': 0.02, 'Gas': 0.05, 'Dining': 0.02, 'Travel': 0.03, 'Entertainment': 0.01},
    'Card7': {'Groceries': 0.03, 'Gas': 0.03, 'Dining': 0.04, 'Travel': 0.01, 'Entertainment': 0.02},
    'Card8': {'Groceries': 0.02, 'Gas': 0.02, 'Dining': 0.03, 'Travel': 0.04, 'Entertainment': 0.05},
    'Card9': {'Groceries': 0.01, 'Gas': 0.03, 'Dining': 0.05, 'Travel': 0.02, 'Entertainment': 0.02},
    'Card10': {'Groceries': 0.04, 'Gas': 0.02, 'Dining': 0.01, 'Travel': 0.05, 'Entertainment': 0.01},
    'Card11': {'Groceries': 0.03, 'Gas': 0.01, 'Dining': 0.02, 'Travel': 0.03, 'Entertainment': 0.04},
    'Card12': {'Groceries': 0.02, 'Gas': 0.02, 'Dining': 0.04, 'Travel': 0.01, 'Entertainment': 0.03},
}

spending = {
    'Groceries': 500,
    'Gas': 300,
    'Dining': 200,
    'Travel': 4000,
    'Entertainment': 250,
    # 'Shopping': 300,
    # 'Utilities': 150,
    # 'Online Subscriptions': 100,
    # 'Healthcare': 200,
    # 'Home Improvement': 150
}

annual_fees = {
    'Card1': 95,
    'Card2': 0,
    'Card3': 85,
    'Card4': 90,
    'Card5': 0,
    'Card6': 70,
    'Card7': 60,
    'Card8': 55,
    'Card9': 95,
    'Card10': 100,
    'Card11': 75,
    'Card12': 80
}

In [80]:
chosen_cards, total_reward, unique_cards = optimize_credit_cards(cards, spending, annual_fees)

print("Chosen Cards by Category:", chosen_cards)
print("Total Reward:", total_reward)
print("Unique Cards Selected:", unique_cards)

Chosen Cards by Category: {'Groceries': 'Card5', 'Gas': 'Card2', 'Dining': 'Card2', 'Travel': 'Card8', 'Entertainment': 'Card5'}
Total Reward: 150.5
Unique Cards Selected: {'Card8', 'Card2', 'Card5'}


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


## Load credit card rewards data set

In [81]:
def import_csv_to_cards_dict(csv_file_path: str) -> Dict:
    """
    This function imports credit card data from a CSV file and formats it into a dictionary.

    Args:
        csv_file_path (str): Path to the CSV file containing credit card data.

    Returns:
        dict: A dictionary where keys are card names and values are dictionaries containing:
              - 'annual_fee' (float): The annual fee of the card.
              - 'rewards' (dict): A dictionary where keys are spending categories
                                  (assuming columns 5 to 14 in the CSV) and values are
                                  the reward points earned per dollar spent in that category.
    """

    # Read the CSV file using Pandas
    credit_card_data = pd.read_csv(csv_file_path)

    # Extract spending categories (assuming columns 5 to 14)
    categories = list(credit_card_data.columns[5:15])

    # Create the formatted card data dictionary
    cards_data = {}
    for index, row in credit_card_data.iterrows():
        card_name = row['Credit Card']
        annual_fee = row['Annual Fee']
        point_value = row['Point Value']
        rewards = {category: row[category] * point_value for category in categories}
        cards_data[card_name] = {'annual_fee': annual_fee, 'rewards': rewards}

    return cards_data


# Load Credit Card Database

In [82]:
filename = 'Credit Card Rewards - Sheet1 (1).csv'

cards_data = import_csv_to_cards_dict(filename)

In [83]:
print(cards_data['Chase Sapphire Reserve'])

{'annual_fee': 595.0, 'rewards': {'Flights': 0.075, 'Hotels': 0.15, 'Other Travel': 0.045, 'Dining': 0.045, 'Grocery': 0.015, 'Gas': 0.015, 'Rideshare': nan, 'Online Shopping': 0.015, 'Amazon': 0.015, 'Costco': 0.015}}


# Run Optimization on Actual Card Data [TBD]